# Map Area

New York, NY, United States

- http://www.openstreetmap.org/relation/175905

- https://mapzen.com/data/metro-extracts/metro/new-york_new-york/

I'm Korean and I live in South Korea. And I've never been to US before. I already heard about New York and Manhatten many times, and if I can get a opportunity I want to visit there. Therefore I'm interested to do the OpenStreetMap data case study with New York part of the map and see the results of querying.

# Problems in the map that I decided to solve

After making a small sample size of the New York area and creating database with the sample, I noticed lots of problems in the data. In this section I'm going to show the problems that I decided to solve and discuss here.

- Inconsistent postal codes (“NY 10533”, “08854-8009”, “10314”)


- Incorrect postal codes (New York area zip codes begin with “005”or "063" or "1". However some zip codes start with "07" or "08". And there are even some zip codes which aren't 5 digit number nor zip+4 code.)


- Same value type but different key names("postcode" and "postal_code", "fax" and "Fax")


- Inconsistent phone, fax numbers("(718) 623-9065", "+1 212 690 4000", "718-733-6813", etc)


- Inconsistent website addresses("bk.com", "http://www.nycgovparks.org/parks/Q048/", "www.skyviewonthehudson.com/")

## Postal codes

<div class = 'alert alert-block alert-success'>
- The results of querying the first 2 digits of values of field 'postcode' and their numbers -  
('11', 26811)
('10', 9473)
('07', 68)
('08', 66)
('NY', 8)
('06', 5)
('32', 1)
('56', 1)
</div>

- Most of postcodes start from 10 or 11. 
- And some of them start from 08 or 07 or 06 and they mean New Jersey or Connecticut. I [used](https://www.unitedstateszipcodes.org/ny/) [these](http://www.geonames.org/postal-codes/postal-codes-us.html) [sites](https://en.wikipedia.org/wiki/New_York_metropolitan_area). Therefore I can infer that this New York OSM data has information about not just New York state, but New York Metropolitan area.

<div class = 'alert alert-block alert-success'>
- The results of querying the values of 'postcode's which are not 5 digit number and their numbers -  
('NY 10533', 6)
('08901-8541', 2)
('07501-210', 1)
('08854-8009', 1)
('08854-8035', 1)
('08854-8037', 1)
('08854-8039', 1)
('08854-8052', 1)
('08901-1004', 1)
('08901-1904', 1)
('08901-2867', 1)
('08901-8502', 1)
('08901-8524', 1)
('08901-8528', 1)
('08901-8539', 1)
('08901-8553', 1)
('10016-0122', 1)
('115422', 1)
('320', 1)
('56', 1)
('NY 10001', 1)
('NY 10026', 1)
</div>

- There are 3 different types of postal codes. First, 5 digit code. Second, 5 digit code added by 4 digits connected by '-'. Third, 5 digit code started by 'NY '.I think that I can programmatically unify them. I'm going to use 5 digit code and zip+4 code both because zip+4 code is better at mailing([reference](https://smartystreets.com/articles/zip-4-code)).
- There are 2 or 3 or 6 digit code and I think that they were input errors. When cleaning the data before sending into DB, I'll remove them.

## Inconsistent phone and fax numbers

<div class = 'alert alert-block alert-success'>
- The results of querying the values of field 'fax' -  
('+1 212 242 1278',)
('+1 212 678 6842',)
('+1 732 5432409',)
('+1 973 761 9193',)
('+1-212-268-0847',)
('718-268-7382',)
('718-361-2734',)
</div>

- I'm going to make the fax numbers have the [same format](http://wiki.openstreetmap.org/wiki/Key:phone) as phone number's.

<div class = 'alert alert-block alert-success'>
- A part of results of querying the values of field 'phone' -  
('(973) 564-9092',)
('+1 (201) 398-0944',)
('+1 203 863 3000',)
('+1 212-254-1675',)
('+1 516 783 86 60',)
('+1-203-703-9080',)
('+17188810010',)
('1-855-928-8672',)
('201 224 8444',)
('201-330-1416',)
</div>

- There are lots of different types of phone numbers in the result. I'm going to make them have ['(NPA) NXX-XXXX' format](https://en.wikipedia.org/wiki/National_conventions_for_writing_telephone_numbers) following the US national telephone writing convention, where NPA is the numbering plan area code and NXX-XXXX is the subscriber number.

## Inconsistent website addresses

<div class = 'alert alert-block alert-success'>
- The results of querying the values of field 'website' which don't start with 'http' -  
('baileyarboretum.org',)
('bigtreebottles.com',)
('bk.com',)
('brookwoodatbayshore.com',)
('hsbc.com',)
('learnandplayny.com',)
('nycgovparks.org',)
('subway.com',)
('usps.com',)
('www.breadsbakery.com',)
('www.creek.net',)
('www.dallasbbq.com',)
('www.efanyc.org',)
('www.mcstudios.com/',)
('www.naturino.com',)
('www.skyviewonthehudson.com/',)
('www.westcaldwellsaladworks.com',)
</div>

- There are three different forms of website addresses. First, URL started from 'http' or 'https' scheme. Second, domain name started from 'www'. Third, domain name without 'www'. 
- I think that it is better to make all website addresses which don't start with 'http' or 'https' start with 'http://' for the consistency(178 of them have this format). And the other reason is that even if there are some sites which don't use 'www' subdomain, there isn't any websites which can be accessed without using http protocol. Referenced [these](https://en.wikipedia.org/wiki/URL) [sites](https://stackoverflow.com/questions/8951423/html-links-without-http-protocol).

## Same value type but different key names

<div class = 'alert alert-block alert-success'>
- The results of querying the values of field 'postal_code' -  
(11696141, 'postal_code', '07933', 'regular')
(11697677, 'postal_code', '07933', 'regular')
(462441978, 'postal_code', '06905', 'regular')
</div>

- Most of postal codes are in the key named 'postcode'. But there are 3 postal codes that are in the key named 'postal_code'. It is similar to the 'fax' and 'Fax' problem, and I'm going to alter the key 'postal_code' to 'postcode'.

<div class = 'alert alert-block alert-success'>
- The results of querying the values of field 'Fax' -  
('732-548-8887',)
</div>

- 'Fax' and 'fax' are same but divided by capital or small letter and the results are like phone number's. I'm going to alter the key 'Fax' to 'fax' and make the numbers have the [same format](http://wiki.openstreetmap.org/wiki/Key:phone) as phone number's.

# How did I solve above problems

## Postal codes

In [ ]:
if key in ['postcode', 'postal_code']:
    if key == 'postal_code':
        key = 'postcode'   
    if value[:2].isdigit() == False:
        value = value[2:].strip()
    if (len(value) == 5 and value.isdigit()) == False and \
        (len(value) == 10 and value[:5].isdigit() and \
         value[-4:].isdigit() and value[5] == '-') == False:
            continue

- First, I changed the key name 'postal_code' to 'postcode' for consistency. And then, if the first two digits are not numbers like 'NY', I removed them and extra spaces. After that, I only accepted the value whose format is 5 digit number or the 10 figures of zip+4 code.

## Inconsistent phone and fax numbers

In [ ]:
if key in ['phone', 'fax', 'Fax']:
    if key == 'Fax':
        key = 'fax'
    if re.search(r'[a-z|A-Z]', value):
        continue
    if value[:2] in ['+1', '01', '1 ', '1-']:
        value = value[2:]
    for ch in [' ', '-', '.', '(', ')']:
        if ch in value:
            value = value.replace(ch, '')
    if (len(value) == 10 and value.isdigit()) == False:
        continue
    value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])

- First, I changed the key name 'Fax' to 'fax' for consistency. And then, I removed the key and value if there is any alphabet in the value using re module and 'continue'. If the first two figures are '+1' or '01' or '1 ' or '1-', I removed the part to make '(xxx) xxx - xxxx' format. And I temporarily removed ' ', '-', '(', ')' to check whether the remained part of the value is 10 digit number. If not, I removed the key and value again. Finally, I changed the value to make it follow the phone number format.

## Inconsistent website addresses

In [ ]:
if key == 'website':
    if value[:4] != 'http':
        value = 'http://' + value

- If the value doesn't start with 'http', I added 'http://'.

## Same value type but different key names

- Before entered into cleaning process of postcode and fax number, I added "**if key == 'A': key = 'B'**" sentence to change key name 'A' to 'B'.

# Additional problems in the map

## Maxspeed

<div class = 'alert alert-block alert-success'>
- The results of querying the values of field 'maxspeed' -  
<dr> 
<dr> 
('10',)
('10 mph',)
('110 mph',)
('125 mph',)
('15',)
('15 mph',)
('160',)
('177',)
('20 mph',)
('201',)
('217',)
('25 mph',)
('30',)
('30 mph',)
('35',)
('35 mph',)
('40 mph',)
('45 mph',)
('48',)
('50',)
('50 mph',)
('55 mph',)
('60',)
('60 mph',)
('64',)
('65 mph',)
('70 mph',)
('72',)
('75 mph',)
('80',)
('90 mph',)
('Amtrak ETT 2010-01-18',)
('Conrail ETT 2013-07-01',)
('Conrail ETT 2013-10-17',)
('signals',)
</div>

- Some of them are just number and others are number added by 'mph'. According to the [wiki](http://wiki.openstreetmap.org/wiki/Key:maxspeed), 'km/h(or kph)' is the basic unit of maxspeed. And the value which not measured by kph should be added to the end of it. 


- But when I see the numbers which aren't added by any unit, some of them are finished by '5' or '0', and others are finished by '1', '2', '7', etc. I think that it is reasonable to think that the former numbers' unit is 'mph', the [US standard speed unit](https://en.wikipedia.org/wiki/Miles_per_hour), and the latter numbers are converted to 'km/h' from 'mph'(because 48, 64, 72, 177, 201, 217 km/h can be converted to 30, 40, 45, 110, 125, 135 mph each). But it is my guess and to correctly add units, I need to find the maxspeed information for each place. Approximate speed limits in New York is in [here](https://en.wikipedia.org/wiki/Speed_limits_in_the_United_States_by_jurisdiction#New_York).


- And some values are about maxspeed's source, not about speed limit. They entered into here because when I process the keys which have lower colons in them, I divided them by ':' and the left part became types and the right part became keys. Therefore in this case of database, I need to use "where tags.key = 'maxspeed' and tags.type = 'regular'" in the query to extract only speed limit information.

## Opening_hours

<div class = 'alert alert-block alert-success'>
- A part of results of querying the values of field 'opening_hours'-  
('24/7',)
('5pm - 9pm (weekdays) , 2pm - 8pm ( weekend)',)
('772',)
('Mon - Fri, 6am - 9pm; Sat, 6am - 8am; Sun, 6am - 7pm',)
('open until 4am',)
</div>

- There are some rules to write opening hours data in the [wiki](http://wiki.openstreetmap.org/wiki/Key:opening_hours). But we can find that there are some datas which don't follow the rules. In the results, some of them use 'am' or 'pm' instead of 'hh:mm' syntax. Also, some of them use words like 'until' and 'to' when it is possible to use '-' symbol. It can be a small problem when users read the datas, but it can be a big problem to classify them programmatically. It is a consistency matter.

# Ideas for improving the data

# Data overview and additional data exploration

## File sizes

New_York_sample.osm : 92 MB
osm_samples.db : 102 MB
sample_nodes.csv : 33 MB
sample_nodes_tags.csv : 1 MB
sample_ways.csv : 4 MB
sample_ways_nodes.csv : 11 MB
sample_ways_tags.csv : 9 MB

I'm going to explore about building, sport, and shop fields. And I'm going to query at nodes_tags and ways_tags tables. 

## Top 5 usage of buildings.

In [2]:
building_query = "select tags.value, count(*) as num \
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'building' and tags.value != 'yes' \
        group by tags.value \
        order by num desc \
        limit 5;"

('house', 5393)
('garage', 4088)
('shed', 489)
('commercial', 317)
('service', 218)

- Most of the buildings are for house or garage in New York.

## The number of shop information.

In [4]:
shop_query = "select tags.key, count(*) as num\
        from (select * from nodes_tags union all select * from ways_tags) tags \
        where tags.key = 'shop';"

('shop', 211)

## The number of shop information which have phone number or email or website or facebook.

In [5]:
shop_with_phone_query = "select tags.key, count(*) as num\
        from (select * from nodes_tags union all select * from ways_tags) tags \
        join (select distinct(id) from nodes_tags \
                where key in ('phone', 'email', 'website', 'facebook') \
                    union all \
              select distinct(id) from ways_tags \
                where key in ('phone', 'email', 'website', 'facebook')) ids \
        on tags.id = ids.id \
        where tags.key = 'shop';"

('shop', 40)

- The number of shop information in New York OSM sample is 211 but only 40 of them have contact information. I think that it is better to make each shop information to have at least one contact information in the map.

# Conclusion